In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import folium as fl
import math

%matplotlib inline
matplotlib.style.use('ggplot')

In [ ]:
sharks = pd.read_csv("data/grey_reef_shark_detections.csv")

In [ ]:
sharks.index.size

In [ ]:
sharks.columns

In [ ]:
sharks

In [ ]:
sharks[30:101]

In [ ]:
sharks.head(100)

In [ ]:
sharks.sort_values(by="timestamp").tail(100)

In [ ]:
sharks.species.unique()

In [ ]:
sharks.species = 'Grey Reef Shark'

In [ ]:
sharks.species.unique()

In [ ]:
sharks.release_locality.unique()

In [ ]:
sharks = sharks.drop('release_locality', axis=1)

In [ ]:
sharks

In [ ]:
sharks.release_datetime = sharks.release_datetime.astype(str).str.replace("T",' ')

In [ ]:
sharks['timestamp'] = sharks['timestamp'].map(pd.to_datetime)
sharks['release_datetime'] = sharks['release_datetime'].map(pd.to_datetime)

In [ ]:
sharks['time_since_release'] = sharks['timestamp'] - sharks['release_datetime']

In [ ]:
sharks

In [ ]:
station_counts = sharks.station.value_counts()

In [ ]:
station_counts

In [ ]:
fig = plt.figure(figsize=(22,10))
station_counts.plot(kind='bar', color='DarkOrange')

In [ ]:
station_counts_df = pd.DataFrame(station_counts).reset_index()
station_counts_df.columns = ['station','count']
station_counts_df

In [ ]:
shark_station_counts = pd.merge(station_counts_df, sharks,  on='station', how='inner')
shark_station_counts = shark_station_counts[['station', 'count', 'longitude', 'latitude']].drop_duplicates().reset_index(drop=True)

In [ ]:
shark_station_counts

In [ ]:
shark_map = fl.Map(location=[shark_station_counts.latitude.median(), shark_station_counts.longitude.median()], 
                tiles='cartodb positron', attr='ESRI', zoom_start=12)

# Add station markers to map
for index, station in shark_station_counts.iterrows():

    #popup string
    station_popup = "%s : %s" % (station['station'], station['count'])

    fl.RegularPolygonMarker([station['latitude'],station['longitude']], 
                    radius=max((station['count']/shark_station_counts['count'].max().astype(float))*15, 3),
                    fill_color='DarkRed',
                    fill_opacity=.3,
                    color='DarkRed',
                    weight=1,
                    number_of_sides=50,
                    popup=station_popup).add_to(shark_map)
shark_map